# 🌱 The following notebook is a simple implementation of a Gradient Boosted Decision Tree using LightGBM.
This Notebooks explains my workflow to implement a ML model from basic or a starting configuration to a more complex implementation </br> 
I'm thinking maybe I will need differenent notebooks to avoid confusion explaining other topics, anyways below the steps for a simple implementation

**Notebook steps**
1. **[Loading Libraries](#1):** Load all the nesessary python modules. 
2. **[Loading the information into DataFrames](#2):** Take the CSV information provided and creates a Pandas Dataframe.
3. **[Exploring the train & target variable](#3):** Review the distribution of the targets we are trying to predict.
4. **[Feature Engineering](#4):** Build some simple features to demonstrate the process.
5. **[Normalizing the training data](#5):** Normalizing the feature to improve the training process.
6. **[Setting the Cross Validation strategy](#6):** Simple separation of the dataset into train and test datasets.
7. **[Training a Machine Learning Model, LGBM](#7):** Construction of a plain LGBM model, Utilize Accuracy to evaluate the model performance.
    * [7.1 Training the Model using a a simple 80/20 train, validation split](#8)
    * [7.2 Validating Model results](#9)
8. **[Training a Machine Learning Model, LGBM](#10):** Construction of a plain LGBM model, Utilize Accuracy to evaluate the model performance.
    * [8.1 Training the Model using a five (5) fold cross validation loop](#11)
    * [8.2 Validating CV Model results](#12)
9. **[Submitting the predictions](#13):** Submit the test dataset prediction for evaluation.
10. **[Document Results](#14):** In this step I document the Notebook score vs. the LB score as the changes made to the model.

I tried to ducument my functions using the `Google Style Python Docstrings` and explain majority of my code with comments </br>
I also add the `%%time` wrapper to all my cells to understand how much time the code is taking to execute and improve in future iterations.


---

**Model Objective:** </br> 
In this competition we are trying to classy the type of cover, based on terrain, soil properties and other parameters, The datasets contains the following variables based on the competion description...

**My Strategy:** </br> 
I really like to start simple and build my models from scratch using other notebooks inspirations, adapting some of the code or my code to something that's easy to understand or follow my logic.
___

**Dataset Description**
This dataset description is from the original competition used to generate the information...

The study area includes four wilderness areas located in the Roosevelt National Forest of northern Colorado. Each observation is a 30m x 30m patch. You are asked to predict an integer classification for the forest cover type. The seven types are:

1. Spruce/Fir
2. Lodgepole Pine
3. Ponderosa Pine
4. Cottonwood/Willow
5. Aspen
6. Douglas-fir
7. Krummholz

The training set (15120 observations) contains both features and the Cover_Type. The test set contains only the features. You must predict the Cover_Type for every row in the test set (565892 observations).

The study area includes four wilderness areas located in the Roosevelt National Forest of northern Colorado. Each observation is a 30m x 30m patch. You are asked to predict an integer classification for the forest cover type. The seven types are:

1. Spruce/Fir
2. Lodgepole Pine
3. Ponderosa Pine
4. Cottonwood/Willow
5. Aspen
6. Douglas-fir
7. Krummholz

The training set (15120 observations) contains both features and the Cover_Type. The test set contains only the features. You must predict the Cover_Type for every row in the test set (565892 observations).

**Data Fields**
* Elevation - Elevation in meters
* Aspect - Aspect in degrees azimuth
* Slope - Slope in degrees
* Horizontal_Distance_To_Hydrology - Horz Dist to nearest surface water features
* Vertical_Distance_To_Hydrology - Vert Dist to nearest surface water features
* Horizontal_Distance_To_Roadways - Horz Dist to nearest roadway
* Hillshade_9am (0 to 255 index) - Hillshade index at 9am, summer solstice
* Hillshade_Noon (0 to 255 index) - Hillshade index at noon, summer solstice
* Hillshade_3pm (0 to 255 index) - Hillshade index at 3pm, summer solstice
* Horizontal_Distance_To_Fire_Points - Horz Dist to nearest wildfire ignition points
* Wilderness_Area (4 binary columns, 0 = absence or 1 = presence) - Wilderness area designation
* Soil_Type (40 binary columns, 0 = absence or 1 = presence) - Soil Type designation
* Cover_Type (7 types, integers 1 to 7) - Forest Cover Type designation

**The wilderness areas are:**

1. Rawah Wilderness Area
2. Neota Wilderness Area
3. Comanche Peak Wilderness Area
4. Cache la Poudre Wilderness Area

**The soil types are:**

1. Cathedral family - Rock outcrop complex, extremely stony.
2. Vanet - Ratake families complex, very stony.
3. Haploborolis - Rock outcrop complex, rubbly.
4. Ratake family - Rock outcrop complex, rubbly.
5. Vanet family - Rock outcrop complex complex, rubbly.
6. Vanet - Wetmore families - Rock outcrop complex, stony.
7. Gothic family.
8. Supervisor - Limber families complex.
9. Troutville family, very stony.
10. Bullwark - Catamount families - Rock outcrop complex, rubbly.
11. Bullwark - Catamount families - Rock land complex, rubbly.
12. Legault family - Rock land complex, stony.
13. Catamount family - Rock land - Bullwark family complex, rubbly.
14. Pachic Argiborolis - Aquolis complex.
15. unspecified in the USFS Soil and ELU Survey.
16. Cryaquolis - Cryoborolis complex.
17. Gateview family - Cryaquolis complex.
18. Rogert family, very stony.
19. Typic Cryaquolis - Borohemists complex.
20. Typic Cryaquepts - Typic Cryaquolls complex.
21. Typic Cryaquolls - Leighcan family, till substratum complex.
22. Leighcan family, till substratum, extremely bouldery.
23. Leighcan family, till substratum - Typic Cryaquolls complex.
24. Leighcan family, extremely stony.
25. Leighcan family, warm, extremely stony.
26. Granile - Catamount families complex, very stony.
27. Leighcan family, warm - Rock outcrop complex, extremely stony.
28. Leighcan family - Rock outcrop complex, extremely stony.
29. Como - Legault families complex, extremely stony.
30. Como family - Rock land - Legault family complex, extremely stony.
31. Leighcan - Catamount families complex, extremely stony.
32. Catamount family - Rock outcrop - Leighcan family complex, extremely stony.
33. Leighcan - Catamount families - Rock outcrop complex, extremely stony.
34. Cryorthents - Rock land complex, extremely stony.
35. Cryumbrepts - Rock outcrop - Cryaquepts complex.
36. Bross family - Rock land - Cryumbrepts complex, extremely stony.
37. Rock outcrop - Cryumbrepts - Cryorthents complex, extremely stony.
38. Leighcan - Moran families - Cryaquolls complex, extremely stony.
39. Moran family - Cryorthents - Leighcan family complex, extremely stony.
40. Moran family - Cryorthents - Rock land complex, extremely stony.
---

**My Updates:** </br>
* 12/05/2021: Created the beginner notebook with simple 80/20 model.
* 12/11/2021: Added a cross validation loop to the model and comparisons against the baseline.
* 12/11/2021: Created more features and corrected some of the dataset ranges, to improve model performance.
* 12/12/2021: Improved the notebook structure, added index and majority of the functions.

**Future Ideas:**
* 12/12/2021: Add Support to track experiments using Neptune.AI (Working On)


**Inspiration:**
Thanks to all the autors of the following notebooks, I used some of the ideas and concepts construct this version of the analysis.
* https://www.kaggle.com/c/tabular-playground-series-dec-2021/discussion/293373 (Feature Engineering Ideas)
* https://www.kaggle.com/c/tabular-playground-series-dec-2021/discussion/293612 (Feature Engineering Ideas)
* https://www.kaggle.com/odins0n/tps-dec-eda-modeling#Table-of-Contents (Visualizations)
* https://sphinxcontrib-napoleon.readthedocs.io/en/latest/example_google.htmlhttps://sphinxcontrib-napoleon.readthedocs.io/en/latest/example_google.html (Docstrings)

<a id='1'></a>
# 1.0- Loading the Requiered Libraries and Configurations.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%time
# Import essential libraries for the modeling process.

# Import visualization libraries...
import matplotlib.pyplot as plt
import plotly.express as px

# Import the lighgbm model library...
from lightgbm import LGBMClassifier

# Import sklearn train, test and split + evaluation metrics.
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

# Import libraries to maintain the notebook performance.
import gc

# Tracking experiments (Future notebook releases).
# import neptune.new as neptune.


In [ ]:
%%time
# I like to disable my Notebook Warnings.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%time
# Define some of the notebook parameters for future experiment replication.
SEED   = 42

In [ ]:
%%time
# Configure notebook display settings to only use 2 decimal places, tables look nicer.
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 15) 
pd.set_option('display.max_rows', 25)

<a id='2'></a>
# 2.0- Loading the Train, Test and Submission Datasets.

In [ ]:
%%time
# Function to built a dataframe from a CSV.
def import_csv(path):
    """
    Import CSV to a dataframe using the specified path...
    """
    df = pd.read_csv(path)
    return df

In [ ]:
%%time
# Invoke the 'import_csv' funtion to create the train_df dataframe.
TRAIN_PATH = '/kaggle/input/tabular-playground-series-dec-2021/train.csv'
train_df = import_csv(TRAIN_PATH)

In [ ]:
%%time
# Model Predictions for the Test Dataset and Submission for Scoring.
TEST_PATH = '/kaggle/input/tabular-playground-series-dec-2021/test.csv'
test_df = import_csv(TEST_PATH)

SUB_PATH = '/kaggle/input/tabular-playground-series-dec-2021/sample_submission.csv'
submission_df = import_csv(SUB_PATH)

In [ ]:
%%time
# Define the memory optimization funtion and simplify the dataframes.
def reduce_mem_usage(df, verbose = True):
    """
    Takes an input dataframe and optimize the variable types to reduce memory consumption.
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2

    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem)) 
        
    return df

In [ ]:
%%time
# Define the memory optimization funtion and simplify the dataframes.
train_df = reduce_mem_usage(train_df)

In [ ]:
%%time
# Define the memory optimization funtion and simplify the dataframes.
test_df = reduce_mem_usage(test_df)

___

<a id='3'></a>
# 3.0- Explore the Train Dataset and Target Variable.

In [ ]:
%%time
# Review the first 5 rows on the dataset to get an idea what we are getting into.
train_df.head()

In [ ]:
%%time
# Some statistical metrics.
train_df.describe()

In [ ]:
%%time
# Trying to understand better the Hill Shade features.
train_df[['Hillshade_9am','Hillshade_Noon','Hillshade_3pm']].describe()

In [ ]:
%%time
# Trying to understand Soil_Type15 and Soil_Type7, 
# The model think are key to predict but it's the same value for all the.
train_df[['Soil_Type15','Soil_Type7']].describe()

In [ ]:
%%time
# Print all the variables,described in the dataset, it's just easy to read them this way, to generate more features.
for feat in train_df.columns:
    print(feat)

In [ ]:
%%time
# Display some metrics of the dataset, will help to understand memory requirements.
train_df.info(max_cols = 15)

In [ ]:
%%time
# Identifying if there is any missing information that needs to be fixed.
train_df.isnull().sum()

**Notes:** There is no **Null** or **NaN** values in the dataset, this make it easier to start training a model without gettin into filling missing values...

In [ ]:
%%time
# Creates a categorical plot for the target variable, help with the visualization of the target distribution.
target_df = pd.DataFrame(train_df['Cover_Type'].value_counts()).reset_index()
target_df.columns = ['Cover_Type', 'count']
fig = px.bar(data_frame =target_df, x = 'Cover_Type', y = 'count', color = "count", color_continuous_scale = "Emrld", width=700, height=400)
fig.show()

In [ ]:
%%time
# Review how many samples of if class there is available in the dataset.
target_df.groupby('Cover_Type').sum().reset_index()


**Notes:** The target distribution in the dataset is unbalanced, we will train a baseline model and then build small improvements over time, in the following sections...

<a id='4'></a>
# 4.0- Feature Engineering.
In this section we will built some simple function, we will construct everything inside funtions for easy appliucation in the future of what we really need, this way the code is more modular

In [ ]:
%%time
# Define fuctions to construct new features for the model.

def euclidean_dist(df, horz_dist, vert_dist, feature_name):
    """
    Calculates the Euclidean distance based on horz. and vert. distance...
    Args:
        df (DataFrame)    : Input dataframe to add the new feature.
        horz_dist (str)   : Name of the horizontal distance field in the dataframe.
        vert_dist (str)   : Name of the vertical distance field in the dataframe.
        feature_name (str): Name of the new created field in the dataframe.
    
    Returns:
        df (DataFrame)    : Dataframe populated with the new feature.
    """
    
    df[feature_name] = np.sqrt(df[horz_dist] ** 2 + df[vert_dist] ** 2)
    return df


def correct_azimut(df):
    """
    Corrects the azimut to be withing 0 - 360 degrees
    Args:
        df (DataFrame): Input dataframe that we need to correct.
    
    Returns:
        df (DataFrame): Dataframe with the feature ranges corrected.
    """
    
    df["Aspect"][df["Aspect"] < 0] += 360
    df["Aspect"][df["Aspect"] > 359] -= 360
    return df


def correct_hillshade(df):
    """
    Corrects the Hillshade values from 0 - 255
    Args:
        df (DataFrame): Input dataframe that we need to correct.
    
    Returns:
        df (DataFrame): Dataframe with the feature ranges corrected.
    """
    
    for feat in df.columns:
        if 'Hillshade' in feat:
            train_df.loc[train_df[feat] < 0, feat] = 0
            train_df.loc[train_df[feat] > 255, feat] = 255
    return df


def count_diversity(df, feature_group_name = 'soil_type'):
    """
    Counts the diversity for example different types of soils in the training row
    Args:
        df (Dataframe): Input dataframe for the feature creation.
        feature_group_name (str): Name of the group in the dataframe that we will be counting.
        
    Returns:
        df (Dataframe): Dataframe populated with the new feature.
    """
    
    features_group = [x for x in df.columns if x.startswith(feature_group_name)]
    df[feature_group_name + "_Count"] = df[features_group].sum(axis=1)
    return df


def remove_cover_type(df, cover_value = 5):
    """
    Remove the selected cover type, Because min. amount of train information.
    
    Args:
        df (Dataframe): Input dataframe to be processed.
        cover_value (int): Value of the cover that needs to be removed from the DataFrame.
    
    Returns:
        df (Dataframe): Dataframe without the selected cover_value.
    """
    
    df = df[df['Cover_Type'] != cover_value]
    return df

In [ ]:
%%time
# Build new features for the model. (Training Set).
train_df = euclidean_dist(train_df, 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Euclidean_Distance_To_Hydrology')
# Improve the values of some of the futures for the model.
train_df = correct_azimut(train_df)
train_df = correct_hillshade(train_df)
# Build other features.
train_df = count_diversity(train_df, feature_group_name = 'Soil_Type')
train_df = count_diversity(train_df, feature_group_name = 'Wilderness_Area')

# Remove conver_type = 5 because there is only 1 example.
train_df = remove_cover_type(train_df)

In [ ]:
%%time
# Build new features for the model (Test Set).
test_df = euclidean_dist(test_df, 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Euclidean_Distance_To_Hydrology')
# Improve the values of some of the futures for the model.
test_df = correct_azimut(test_df)
test_df = correct_hillshade(test_df)
# Build other features.
test_df = count_diversity(test_df, feature_group_name = 'Soil_Type')
test_df = count_diversity(test_df, feature_group_name = 'Wilderness_Area')

In [ ]:
%%time
# Review some of the features created in the previous step.
train_df.sample(10)

___

<a id='5'></a>
# 5.0- Preparing the Dataset for the Training Step.
In this section I organize the dataset for the training funtions, for example I will create a list of the futures that are important as identify the training variable. </br>
I will also focus on identifying categorical vs numerical features to improve the model performance as also apply some normalization to the numerical variables.

In [ ]:
%%time
# Prepare the dataset for training, remove some of the not valuable variables.
remove = ['Id', 'Cover_Type', 'Soil_Type15', 'Soil_Type7'] # Removes the Target from the train dataset.
features = [values for values in train_df.columns if values not in remove]
X = train_df[features]
y = train_df['Cover_Type']

X_test = test_df[features]

In [ ]:
%%time
# Identify categorical and numerical features.
NUMBER_OF_UNIQUE = 25
categorical_features = [col for col in features if train_df[col].nunique() < NUMBER_OF_UNIQUE]
numerical_features = [col for col in features if train_df[col].nunique() >= NUMBER_OF_UNIQUE]

In [ ]:
%%time
# Print a list of all the categorical features identified.
print(categorical_features)

In [ ]:
%%time
# Deletes the original dataframes to save memory
del train_df # Deletes the train_df dataframe
del test_df  # Deletes the test_df dataframe
gc.collect()

In [ ]:
%%time
# Create a funtion to normalize some of the numeric variables.
# ...

<a id='6'></a>
# 6.0- Setting a Simple Cross Validation Strategy 

In [ ]:
# Split the dataset into train and validation for model first calibration...
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = SEED, test_size = 0.2)

---

<a id='7'></a>
# 7.0- Training a Machine Learning Model, LGBMTraining a Machine Learning Model, LGBM

In [ ]:
%%time
# Define some of the model parameters for future experiment replication.
ESTIMATORS = 10 # Default value for the training 2048
LR = 0.1
VERBOSE = 16
EARLY_STOPPING = 250

In [ ]:
%%time
# Define the LGBM most popular model parameters for the training step.

lgb_params = {'n_estimators'     : ESTIMATORS,      # Number of boosting iterations.
              'random_state'     : SEED,            # Random seed initilizer for the model, helps to replicate the experiments.
              'learning_rate'    : LR,              # The model learning rate.
              'subsample'        : 0.95,            # Row subsample from the dataset, like feature_fraction, but this will randomly select part of data without resampling
              'subsample_freq'   : 1,               # Use or not subsample frequency.
              'colsample_bytree' : 0.75,            # LightGBM will randomly select a subset of features on each iteration (tree).
              'reg_alpha'        : 0.5,             # L1 regularization.
              'reg_lambda'       : 0.5,             # L2 regularization.
              'min_child_weight' : 1e-3,            # Minimal sum hessian in one leaf, it can be used to deal with over-fitting.
              'min_child_samples': 32,              # Minimal number of data in one leaf. Can be used to deal with over-fitting.
              'objective'        : 'multiclass',    # Softmax objective function.
              'metric'           : 'multi_logloss', # Log loss for multi-class classification.
              'device_type'      : 'gpu',
             }   

<a id='8'></a>
# 7.1- Training a Simple Model, Gradient Boosted Decision Tree using LightGBM.


In [ ]:
%%time
# Train a LGBM model using the defined parameterst 'lgb_params'
lgb_classifier = LGBMClassifier(**lgb_params)

lgb_classifier.fit(X_train, 
                   y_train, 
                   eval_set = [(X_val, y_val)],
                   categorical_feature = categorical_features,
                   early_stopping_rounds = EARLY_STOPPING, # will stop training if one metric of one validation data doesn’t improve in last early_stopping_round rounds
                   verbose = VERBOSE, # Controls the level of LightGBM’s verbosity
                  )

<a id='9'></a>
# 7.2- Validating Model Results.

In [ ]:
%%time
# Review top 10 features importance for the model.
# Sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(lgb_classifier.feature_importances_,X.columns)), columns=['Value','Feature'])

In [ ]:
%%time
# Review the top five(5) Feature the model believes are important
print(feature_imp.head(10), '\n')
print(feature_imp.tail(10))

In [ ]:
%%time
# Results of the model predictions and evaluation against true target...
preds_valid = lgb_classifier.predict(X_val)
accuracy = accuracy_score(y_val, preds_valid)
print("Mean Accuracy :", accuracy)

* **Mean Accuracy : 0.96136375** ... Simple baseline model, No Features outside the default variables...
* **Mean Accuracy : 0.96140750** ... Added Euclidean distance as a Feature to the model...
* **Mean Accuracy : 0.96140750** ... Added Euclidean distance as a Feature + Corrections to the ranges of the Azimut and the Hillshade...
* **Mean Accuracy : 0.96057125** ... Added Euclidean distance as a Feature + Corrections to the ranges of the Azimut and the Hillshade, New Train Parameters 128 Estimators...
* **Mean Accuracy : 0.96060625** ... Added Euclidean distance as a Feature + Corrections to the ranges of the Azimut and the Hillshade, New Train Parameters 128 Estimators, Removed Soil 17 & 7...
* **Mean Accuracy : 0.96217000** ... Added Euclidean distance as a Feature + Corrections to the ranges of the Azimut and the Hillshade, New Train Parameters 2048 Estimators, Removed Soil 17 & 7...

In [ ]:
%%time
# Deletes the train and validation dataframes created for the simple training to save memory
del X_train, 
del X_val, 
del y_train, 
del y_val
gc.collect()

---

<a id='10'></a>
# 8.0- Training a Machine Learning Model, LGBM

In [ ]:
%%time
# Define some of the cross validation model parameters
FOLDS = 20

<a id='11'></a>
# 8.1- Training a Simple Model in a Cross Validation Loop, Gradient Boosted Decision Tree using LightGBM.

In [ ]:
%%time
# Creates a Cross validation loop

folds = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state = SEED)

oof_preds = np.zeros(X.shape[0])
sub_preds = np.zeros(X_test.shape[0])

for n_fold, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
    trn_x, trn_y = X[features].iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = X[features].iloc[val_idx], y.iloc[val_idx]
    
    clf = LGBMClassifier(**lgb_params)
    clf.fit(trn_x, 
            trn_y, 
            eval_set = [(val_x, val_y)],
            categorical_feature = categorical_features,
            verbose=VERBOSE, 
            early_stopping_rounds=EARLY_STOPPING)
    
    oof_preds[val_idx] = clf.predict(val_x, num_iteration=clf.best_iteration_)
    sub_preds += clf.predict(X_test[features], num_iteration=clf.best_iteration_) / folds.n_splits
    
    print('Fold %2d Accuracy Score: %.6f' % (n_fold + 1, accuracy_score(val_y, oof_preds[val_idx])))
    del trn_x, trn_y, val_x, val_y
    gc.collect()

print('\n')    
print('Avg. Accuracy Score %.6f' % accuracy_score(y, oof_preds))

<a id='12'></a>
# 8.2- Validating CV Model Results.

In [ ]:
%%time
# Review top 10 features importance for the model.
# Sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp_cv = pd.DataFrame(sorted(zip(clf.feature_importances_,X.columns)), columns=['Value','Feature'])

In [ ]:
%%time
# Review the top five(5) Feature the model believes are important
print(feature_imp_cv.head(10), '\n')
print(feature_imp_cv.tail(10))

___

<a id='13'></a>
# 9.0 - Submitting the Predictions to Kaggle...

In [ ]:
%%time
# Calculate the predictions using the 80/20 train / val model...
preds_test = lgb_classifier.predict(X_test[features])

# Submit predicitons from the CV trained Model...
submission_df['Cover_Type'] = preds_test
submission_df.to_csv('simple_sub_v12122021.csv', index = None)

In [ ]:
%%time
# Submit predicitons from the CV trained Model...
sub_preds = sub_preds.astype(np.int)
submission_df['Cover_Type'] = sub_preds
submission_df.to_csv('cv_sub_v12112021.csv', index = None)

<a id='14'></a>
# 10.0- Results Documentation...
I'm planning to add a Google Sheets that containts the results of all the iterations completed with this models as the modification added and ideas for improvement...
...